# Lambda³ Weather Analysis - 自動ペア分析システム

このノートブックは、Lambda³理論に基づいて気象データの構造テンソル相互作用を分析します。

## ワークフロー
1. GitHubリポジトリから必要なコードを取得
2. Open-Meteo APIから東京の気象データを取得
3. Lambda³特徴量を抽出
4. 全パラメータペアの自動分析を実行

## 1. 環境セットアップと依存関係のインストール

In [ ]:
# 必要なライブラリのインストール
!pip install openmeteo-requests requests-cache retry-requests pymc arviz numba networkx -q

print("✓ ライブラリのインストール完了")

## 2. GitHubから Lambda³ コードを取得

In [ ]:
# GitHubリポジトリをクローン
!git clone https://github.com/your-username/bayesian-event-detector.git

# ワーキングディレクトリを設定
import os
os.chdir('bayesian-event-detector/sample')

# 必要なファイルの存在確認
import glob
files = glob.glob('*.py')
print(f"利用可能なPythonファイル: {files}")

# WeatherAnalysis.pyが存在しない場合の対処
if 'WeatherAnalysis.py' not in files:
    print("⚠️ WeatherAnalysis.pyが見つかりません。アップロードしてください。")
else:
    print("✓ WeatherAnalysis.py 確認完了")

## 3. 自動ペア分析モジュールの作成

In [ ]:
# lambda3_auto_pair.py を作成
auto_pair_code = '''# ===============================
# Lambda³ Automatic Pair Analysis System
# ===============================
from itertools import combinations
from typing import Dict, List, Tuple, Optional, Union
import pandas as pd
import numpy as np
from dataclasses import dataclass, field
from pathlib import Path

@dataclass
class PairAnalysisConfig:
    """Configuration for automatic pair analysis."""
    # Analysis parameters
    analyze_all_pairs: bool = True
    max_pairs: Optional[int] = None  # None means analyze all
    min_series_length: int = 100
    
    # Pair filtering criteria
    min_correlation: Optional[float] = None  # Filter pairs by minimum correlation
    exclude_patterns: List[str] = field(default_factory=list)  # Patterns to exclude
    include_only_patterns: List[str] = field(default_factory=list)  # If set, only these
    
    # Analysis depth
    detailed_analysis_limit: int = 5  # Number of pairs for detailed plots
    summary_only_after: int = 10  # Switch to summary mode after this many pairs
    
    # Output configuration
    save_results: bool = True
    output_dir: str = "lambda3_results"
    generate_report: bool = True

class Lambda3AutoPairAnalyzer:
    """Automatic pair analysis system for Lambda³ framework."""
    
    def __init__(self, config: PairAnalysisConfig = None):
        self.config = config or PairAnalysisConfig()
        self.analysis_results = {}
        self.pair_metadata = {}
        
    def detect_data_structure(self, data: Union[pd.DataFrame, Dict[str, np.ndarray]]) -> Dict[str, List[str]]:
        """Automatically detect data structure and categorize columns."""
        if isinstance(data, pd.DataFrame):
            columns = data.columns.tolist()
            numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
        else:
            columns = list(data.keys())
            numeric_cols = columns
        
        # Categorize columns based on patterns
        categories = {
            \'weather\': [],
            \'temperature\': [],
            \'humidity\': [],
            \'pressure\': [],
            \'wind\': [],
            \'precipitation\': [],
            \'other\': []
        }
        
        patterns = {
            \'temperature\': [\'temp\', \'temperature\', \'dewpoint\', \'dew_point\'],
            \'humidity\': [\'humid\', \'rh\', \'relative_humidity\'],
            \'pressure\': [\'pressure\', \'press\', \'hpa\', \'mbar\'],
            \'wind\': [\'wind\', \'gust\', \'speed\'],
            \'precipitation\': [\'precip\', \'rain\', \'snow\', \'precipitation\']
        }
        
        for col in numeric_cols:
            col_lower = col.lower()
            categorized = False
            
            for category, keywords in patterns.items():
                if any(keyword in col_lower for keyword in keywords):
                    categories[category].append(col)
                    categories[\'weather\'].append(col)
                    categorized = True
                    break
            
            if not categorized:
                categories[\'other\'].append(col)
        
        return categories
    
    def generate_pair_list(self, 
                          series_dict: Dict[str, np.ndarray],
                          categories: Optional[Dict[str, List[str]]] = None) -> List[Tuple[str, str]]:
        """Generate intelligent pair list based on data structure."""
        all_series = list(series_dict.keys())
        
        # Apply include/exclude patterns
        if self.config.include_only_patterns:
            all_series = [s for s in all_series 
                         if any(pattern in s.lower() for pattern in self.config.include_only_patterns)]
        
        if self.config.exclude_patterns:
            all_series = [s for s in all_series 
                         if not any(pattern in s.lower() for pattern in self.config.exclude_patterns)]
        
        # Generate all unique pairs
        all_pairs = list(combinations(all_series, 2))
        
        # Apply correlation filter if specified
        if self.config.min_correlation is not None:
            filtered_pairs = []
            for a, b in all_pairs:
                corr = np.corrcoef(series_dict[a], series_dict[b])[0, 1]
                if abs(corr) >= self.config.min_correlation:
                    filtered_pairs.append((a, b))
                    self.pair_metadata[(a, b)] = {\'correlation\': corr}
            all_pairs = filtered_pairs
        
        # Apply max pairs limit
        if self.config.max_pairs and len(all_pairs) > self.config.max_pairs:
            print(f"Limiting analysis to {self.config.max_pairs} pairs out of {len(all_pairs)} total")
            all_pairs = all_pairs[:self.config.max_pairs]
        
        return all_pairs

# Continue with remaining methods...
# (Full implementation would be too long for this cell)
'''

# ファイルに保存
with open('lambda3_auto_pair.py', 'w') as f:
    f.write(auto_pair_code)

print("✓ lambda3_auto_pair.py 作成完了")

## 4. Open-Meteo APIから東京の気象データを取得

In [ ]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry

# APIクライアントのセットアップ
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# API呼び出しパラメータ
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 35.6812,
    "longitude": 139.7671,
    "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", 
               "precipitation", "wind_speed_10m", "surface_pressure"],
    "timezone": "Asia/Tokyo",
    "start_date": "2025-06-20",
    "end_date": "2025-06-27"
}

print("🌐 Open-Meteo APIからデータを取得中...")
responses = openmeteo.weather_api(url, params=params)

# レスポンスの処理
response = responses[0]
print(f"\n📍 座標: {response.Latitude()}°N {response.Longitude()}°E")
print(f"🏔️ 標高: {response.Elevation()} m")
print(f"🕐 タイムゾーン: {response.Timezone()}{response.TimezoneAbbreviation()}")

# 時系列データの処理
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(5).ValuesAsNumpy()

# DataFrameの作成
hourly_data = {
    "date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    ),
    "temperature_2m": hourly_temperature_2m,
    "relative_humidity_2m": hourly_relative_humidity_2m,
    "dew_point_2m": hourly_dew_point_2m,
    "precipitation": hourly_precipitation,
    "wind_speed_10m": hourly_wind_speed_10m,
    "surface_pressure": hourly_surface_pressure
}

hourly_dataframe = pd.DataFrame(data = hourly_data)

# データの概要表示
print("\n📊 取得したデータの概要:")
print(hourly_dataframe.info())
print("\n🔍 最初の5行:")
print(hourly_dataframe.head())

# CSVに保存
hourly_dataframe.to_csv("tokyo_weather_days.csv", index=False)
print("\n✅ CSV出力完了: tokyo_weather_days.csv")

## 5. Lambda³ 特徴量の抽出

In [ ]:
# WeatherAnalysis.pyをインポート
from WeatherAnalysis import (
    L3Config, 
    calc_lambda3_features_v2,
    load_csv_data,
    validate_series_lengths
)

# データの読み込み
print("📂 CSVファイルからデータを読み込み中...")
series_dict = load_csv_data(
    "tokyo_weather_days.csv",
    time_column="date",
    value_columns=["temperature_2m", "relative_humidity_2m", "dew_point_2m", 
                   "precipitation", "wind_speed_10m", "surface_pressure"]
)

# データの検証
series_dict = validate_series_lengths(series_dict)

# Lambda³設定
config = L3Config(
    T=len(next(iter(series_dict.values()))),
    draws=4000,  # Colab用に削減
    tune=4000,
    delta_percentile=95.0
)

print(f"\n⚙️ Lambda³設定:")
print(f"  - 時系列長: {config.T}")
print(f"  - MCMCドロー数: {config.draws}")
print(f"  - ジャンプ検出閾値: {config.delta_percentile}パーセンタイル")

# 各系列の特徴量抽出
print("\n🔬 Lambda³特徴量を抽出中...")
features_dict = {}

for name, data in series_dict.items():
    print(f"\n  処理中: {name}")
    feats = calc_lambda3_features_v2(data, config)
    
    features_dict[name] = {
        'data': data,
        'delta_LambdaC_pos': feats[0],
        'delta_LambdaC_neg': feats[1],
        'rho_T': feats[2],
        'time_trend': feats[3],
        'local_jump': feats[4]
    }
    
    # 統計情報
    n_pos = np.sum(feats[0])
    n_neg = np.sum(feats[1])
    mean_tension = np.mean(feats[2])
    
    print(f"    - 正の構造変化 (∆ΛC+): {n_pos}")
    print(f"    - 負の構造変化 (∆ΛC-): {n_neg}")
    print(f"    - 平均張力スカラー (ρT): {mean_tension:.3f}")

print("\n✅ 特徴量抽出完了")

## 6. 自動ペア分析の実行

In [ ]:
# 簡易版の自動ペア分析実装
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns

# すべての可能なペアを生成
series_names = list(series_dict.keys())
all_pairs = list(combinations(series_names, 2))

print(f"🔗 分析するペア数: {len(all_pairs)}")
print("\nペアリスト:")
for i, (a, b) in enumerate(all_pairs):
    print(f"  {i+1}. {a} ↔ {b}")

# 同期率の計算（簡易版）
from WeatherAnalysis import calculate_sync_profile

sync_results = {}
print("\n📊 同期率分析を実行中...")

for pair in all_pairs:
    a, b = pair
    
    # 正のジャンプイベントの同期率を計算
    sync_profile, max_sync, optimal_lag = calculate_sync_profile(
        features_dict[a]['delta_LambdaC_pos'].astype(np.float64),
        features_dict[b]['delta_LambdaC_pos'].astype(np.float64),
        lag_window=10
    )
    
    sync_results[pair] = {
        'sync_rate': max_sync,
        'optimal_lag': optimal_lag,
        'profile': sync_profile
    }
    
    print(f"  {a} ↔ {b}: σₛ = {max_sync:.3f} (最適ラグ: {optimal_lag})")

# 同期率マトリックスの作成
n = len(series_names)
sync_matrix = np.zeros((n, n))

for i, a in enumerate(series_names):
    for j, b in enumerate(series_names):
        if i == j:
            sync_matrix[i, j] = 1.0
        elif (a, b) in sync_results:
            sync_matrix[i, j] = sync_results[(a, b)]['sync_rate']
        elif (b, a) in sync_results:
            sync_matrix[i, j] = sync_results[(b, a)]['sync_rate']

# ヒートマップの表示
plt.figure(figsize=(10, 8))
sns.heatmap(sync_matrix, 
            annot=True, 
            fmt='.3f',
            xticklabels=series_names,
            yticklabels=series_names,
            cmap='Blues',
            vmin=0, vmax=1,
            square=True,
            cbar_kws={'label': '同期率 σₛ'})

plt.title('気象パラメータ間の構造同期率マトリックス', fontsize=16)
plt.tight_layout()
plt.show()

# トップ同期ペアの表示
sorted_pairs = sorted(sync_results.items(), key=lambda x: x[1]['sync_rate'], reverse=True)

print("\n🏆 トップ5同期ペア:")
for i, ((a, b), data) in enumerate(sorted_pairs[:5]):
    print(f"  {i+1}. {a} ↔ {b}: σₛ = {data['sync_rate']:.3f} (ラグ: {data['optimal_lag']})")

## 7. 詳細な相互作用分析（選択的実行）

In [ ]:
# 最も同期率の高いペアの詳細分析
if sorted_pairs:
    top_pair = sorted_pairs[0][0]
    name_a, name_b = top_pair
    
    print(f"\n🔍 最高同期ペアの詳細分析: {name_a} ↔ {name_b}")
    
    from WeatherAnalysis import (
        fit_l3_bayesian_regression_asymmetric,
        plot_sync_profile
    )
    
    # ベイズ回帰モデルの適合
    print("\n📈 ベイズ回帰モデルを適合中...")
    print("（これには数分かかる場合があります）")
    
    # A に対する B の影響
    trace_a = fit_l3_bayesian_regression_asymmetric(
        data=features_dict[name_a]['data'],
        features_dict={
            'delta_LambdaC_pos': features_dict[name_a]['delta_LambdaC_pos'],
            'delta_LambdaC_neg': features_dict[name_a]['delta_LambdaC_neg'],
            'rho_T': features_dict[name_a]['rho_T'],
            'time_trend': features_dict[name_a]['time_trend']
        },
        config=config,
        interaction_pos=features_dict[name_b]['delta_LambdaC_pos'],
        interaction_neg=features_dict[name_b]['delta_LambdaC_neg'],
        interaction_rhoT=features_dict[name_b]['rho_T']
    )
    
    # 結果のサマリー
    import arviz as az
    summary_a = az.summary(trace_a)
    
    print(f"\n📊 {name_b} → {name_a} の相互作用効果:")
    if 'beta_interact_pos' in summary_a.index:
        print(f"  正の相互作用: β = {summary_a.loc['beta_interact_pos', 'mean']:.3f}")
    if 'beta_interact_neg' in summary_a.index:
        print(f"  負の相互作用: β = {summary_a.loc['beta_interact_neg', 'mean']:.3f}")
    
    # 同期プロファイルの可視化
    plot_sync_profile(
        sync_results[top_pair]['profile'],
        title=f"{name_a} ↔ {name_b} の同期プロファイル"
    )
    
else:
    print("⚠️ 分析可能なペアが見つかりませんでした。")

## 8. 結果の保存とレポート生成

In [ ]:
# 結果をCSVに保存
import pandas as pd
from datetime import datetime

# 同期率結果のDataFrame作成
sync_df = pd.DataFrame([
    {
        'series_a': pair[0],
        'series_b': pair[1],
        'sync_rate': data['sync_rate'],
        'optimal_lag': data['optimal_lag']
    }
    for pair, data in sync_results.items()
])

# ソートして保存
sync_df = sync_df.sort_values('sync_rate', ascending=False)
sync_df.to_csv('lambda3_sync_results.csv', index=False)
print("✅ 同期率結果を保存: lambda3_sync_results.csv")

# マークダウンレポートの生成
report = f"""# Lambda³ Weather Analysis Report
生成日時: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## 分析概要
- 分析期間: 2025-06-20 〜 2025-06-27
- 場所: 東京 (35.6812°N, 139.7671°E)
- 分析パラメータ数: {len(series_names)}
- 分析ペア数: {len(all_pairs)}

## Lambda³ 理論による解釈
気象現象は時間的因果性ではなく、構造テンソル（Λ）の意味空間における
相互作用として理解されます。高い同期率は構造的共鳴を示唆します。

## トップ同期ペア
| 順位 | パラメータ A | パラメータ B | 同期率 σₛ | 最適ラグ |
|------|-------------|-------------|----------|----------|
"""

for i, row in sync_df.head(10).iterrows():
    report += f"| {i+1} | {row['series_a']} | {row['series_b']} | {row['sync_rate']:.3f} | {row['optimal_lag']} |\n"

report += f"""\n## 構造的洞察
最高同期ペア ({sync_df.iloc[0]['series_a']} ↔ {sync_df.iloc[0]['series_b']}) は
σₛ = {sync_df.iloc[0]['sync_rate']:.3f} の強い構造的共鳴を示しています。
これは両パラメータが共通の意味空間で∆ΛC pulsationを共有していることを示唆します。
"""

# レポートを保存
with open('lambda3_analysis_report.md', 'w', encoding='utf-8') as f:
    f.write(report)

print("\n✅ 分析レポートを保存: lambda3_analysis_report.md")
print("\n🎉 Lambda³ 自動ペア分析が完了しました！")

## 9. 結果のダウンロード（Colab用）

In [ ]:
# Google Colabで実行している場合、結果ファイルをダウンロード
try:
    from google.colab import files
    
    print("📥 結果ファイルをダウンロード可能にしています...")
    
    # ダウンロード可能なファイル
    files_to_download = [
        'tokyo_weather_days.csv',
        'lambda3_sync_results.csv',
        'lambda3_analysis_report.md'
    ]
    
    for file in files_to_download:
        try:
            files.download(file)
            print(f"  ✓ {file}")
        except:
            print(f"  ✗ {file} - ファイルが見つかりません")
            
except ImportError:
    print("ℹ️ ローカル環境で実行中です。ファイルは現在のディレクトリに保存されています。")

---

## 📚 補足情報

### Lambda³ Theory について
- すべての現象は構造テンソル（Λ）の相互作用として理解されます
- 時間的因果性は仮定せず、意味空間での構造的共鳴を分析します
- ∆ΛC は構造的変化（ジャンプ）を表します
- ρT（張力スカラー）は局所的な不安定性を示します

### カスタマイズ方法
1. 分析期間の変更: `start_date` と `end_date` を調整
2. 場所の変更: `latitude` と `longitude` を調整
3. パラメータの追加: Open-Meteo APIの `hourly` リストに追加
4. 分析深度の調整: `L3Config` の `draws` と `tune` を調整

### トラブルシューティング
- メモリ不足: `draws` と `tune` を減らす（例: 2000）
- 実行時間が長い: `max_pairs` を設定して分析ペア数を制限
- インポートエラー: 必要なライブラリを再インストール